In [ ]:
# Mount drive

from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from keras.applications import ResNet50
from keras.applications.resnet import preprocess_input
from keras.preprocessing import image
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.preprocessing import image
import os
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit


In [ ]:
# Function to load and preprocess images
def load_and_preprocess_image(img_path, label):
    img = image.load_img(img_path, target_size=(img_width, img_height))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array, label

In [ ]:
# Function to extract features from an image
def extract_features(img_path, label):
    img = image.load_img(img_path, target_size=(img_width, img_height))
    img_array = image.img_to_array(img)
    img_array = preprocess_input(img_array)
    return img_array, label

In [ ]:
# Set the image size according to the ResNet-101V2 input shape
img_width, img_height = 256, 256

In [ ]:
# Set the path to the Google Drive dataset
base_dataset_path = '/content/drive/MyDrive/dataset/dataset'

In [ ]:
batch_size = 15

In [ ]:
# Create a ResNet-50 model with pre-trained weights from ImageNet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

In [ ]:
# Create lists to store image data and labels
image_data = []
image_labels = []

In [ ]:
# Iterate through each mask type
mask_types = ['FFP2 mask', 'cotton mask', 'n95 mask dataset', 'surgical mask dataset']
for label, mask_type in enumerate(mask_types):
    mask_folder = os.path.join(base_dataset_path, mask_type)

    # Load images and extract features
    for filename in os.listdir(mask_folder):
        if filename.endswith(".jpg"):
            img_path = os.path.join(mask_folder, filename)
            img_data, img_label = extract_features(img_path, label)
            image_data.append(img_data)
            image_labels.append(img_label)

# Convert the lists to numpy arrays
image_data = np.array(image_data)
image_labels = np.array(image_labels)


In [ ]:
# Use StratifiedShuffleSplit to split the data
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
for train_index, test_val_index in sss.split(image_data, image_labels):
    train_data, test_val_data = image_data[train_index], image_data[test_val_index]
    train_labels, test_val_labels = image_labels[train_index], image_labels[test_val_index]


In [ ]:
# Further split the test_val set into test and validation sets
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
for test_index, val_index in sss.split(test_val_data, test_val_labels):
    test_data, val_data = test_val_data[test_index], test_val_data[val_index]
    test_labels, val_labels = test_val_labels[test_index], test_val_labels[val_index]


In [ ]:
# Create a ResNet-50 model with pre-trained weights from ImageNet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Add custom layers for your specific classification task
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))  # Add dropout after the first dense layer
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Add dropout after the second dense layer
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='softmax'))

# Freeze the weights of the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(lr=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                        

In [ ]:
# Data augmentation for training images
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2
)

# Rescaling for validation and test images
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Generate the training, validation, and test data using flow_from_directory
train_generator = train_datagen.flow(train_data, train_labels, batch_size=batch_size)
validation_generator = val_test_datagen.flow(val_data, val_labels, batch_size=batch_size, shuffle=False)
test_generator = val_test_datagen.flow(test_data, test_labels, batch_size=batch_size, shuffle=False)


In [ ]:
# Fine-tune the model
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // batch_size
)

<ipython-input-15-99a12a3b4cc1>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/50
23/23 [==============================] - 76s 3s/step - loss: 1.5576 - accuracy: 0.3314 - val_loss: 1.3582 - val_accuracy: 0.2400
Epoch 2/50
23/23 [==============================] - 77s 3s/step - loss: 1.4401 - accuracy: 0.3721 - val_loss: 1.3337 - val_accuracy: 0.4133
Epoch 3/50
23/23 [==============================] - 77s 3s/step - loss: 1.3705 - accuracy: 0.3517 - val_loss: 1.3302 - val_accuracy: 0.4133
Epoch 4/50
23/23 [==============================] - 76s 3s/step - loss: 1.3639 - accuracy: 0.3372 - val_loss: 1.3186 - val_accuracy: 0.4133
Epoch 5/50
23/23 [==============================] - 79s 3s/step - loss: 1.3431 - accuracy: 0.3634 - val_loss: 1.3032 - val_accuracy: 0.4133
Epoch 6/50
23/23 [==============================] - 76s 3s/step - loss: 1.3294 - accuracy: 0.4012 - val_loss: 1.3217 - val_accuracy: 0.4133
Epoch 7/50
23/23 [==============================] - 76s 3s/step - loss: 1.3473 - accuracy: 0.3692 - val_loss: 1.3150 - val_accuracy: 0.4133
Epoch 8/50
23/23 [==

In [ ]:
# Save the model weights
model.save_weights('/content/drive/MyDrive/Dataset/dataset/resnet50_face_mask.h5')


FileNotFoundError: ignored

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(validation_generator)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()


In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()